In [1]:

import json
import os
import polars as pl
from polars import col as c
import polars.selectors as cs
import re
from data_federation.input_model import SmallflexInputSchema
import owncloud
import streamlit as st
from datetime import date
import plotly.graph_objs as go
from data_federation.parser.market_price import parse_market_price
from data_federation.parser.discharge_flow import parse_discharge_flow
from data_federation.parser.weather import parse_weather
from data_federation.parser.aegina_hydro_power_plant import parse_aegina_hydro_power_plant
from data_federation.parser.merezenbach_hydro_power_plant import parse_merezenbach_hydro_power_plant
from data_federation.parser.morel_hydro_power_plant import parse_morel_hydro_power_plant
from data_federation.parser.aegina_wind_power_plant import parse_aegina_wind_power_plant
from data_federation.parser.parser_pipeline import input_pipeline
from utility.general_function import dictionary_key_filtering, scan_switch_directory, modify_string, generate_uuid, pl_to_dict
from utility.polars_operation import modify_string_col, generate_uuid_col
from data_display.input_data_plots import plot_market_prices
from data_federation.parser.hydro_power_plant import get_hydro_power_plant_data
from config import settings
import plotly.express as px
from datetime import timedelta, date
from utility.pyomo_preprocessing import *
from datetime import timedelta, date, datetime, timezone
from datetime import timedelta, date
import numpy as np
from typing_extensions import Optional
import pyomo.environ as pyo

from itertools import product

os.chdir(os.getcwd().replace("/src", ""))


['/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/home/ltomasini/git/smallflex/smallflex-vpp-codes/.venv/lib/python3.12/site-packages']


In [2]:

input_file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))
market_price_metadata: dict = json.load(open(input_file_names["market_price_metadata"]))


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))
small_flex_input_schema: SmallflexInputSchema = SmallflexInputSchema()\
    .duckdb_to_schema(file_path=output_file_names["duckdb_input"])


Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 12/12 [00:01<00:00,  9.04it/s]


In [4]:
year = 2020

min_datetime = datetime(year, 1, 1, tzinfo=timezone.utc)
max_datetime = datetime(year+1, 1, 1, tzinfo=timezone.utc)
first_time_delta = timedelta(days=7)
second_time_delta = timedelta(minutes=60)
n_segments = 5

market_country = "CH"
market = "DA"
hydro_power_plant_name = "Aegina hydro"

power_plant_metadata = small_flex_input_schema.hydro_power_plant\
    .filter(c("name") == hydro_power_plant_name).to_dicts()[0]
water_basin_uuid = power_plant_metadata["upstream_basin_fk"]
power_plant_uuid = power_plant_metadata["uuid"]

first_datetime_index, second_datetime_index = generate_datetime_index(
    min_datetime=min_datetime, max_datetime=max_datetime, first_time_delta=first_time_delta, second_time_delta=second_time_delta)

market_price_measurement:pl.DataFrame = small_flex_input_schema.market_price_measurement\
    .filter(c("country") == market_country).filter(c("market") == market)
    
discharge_flow_measurement: pl.DataFrame = small_flex_input_schema.discharge_flow_measurement\
    .filter(c("river") == "Griessee")\
    .with_columns(
        (c("value") * timedelta(hours=1).total_seconds()).alias("discharge_volume")
    )
    

basin_metadata = small_flex_input_schema.water_basin.filter(c("uuid") == water_basin_uuid).to_dicts()[0]

down_stream_height = small_flex_input_schema.water_basin.filter(c("uuid") == power_plant_metadata["downstream_basin_fk"])["height_max"][0]
basin_height_volume_table: pl.DataFrame = small_flex_input_schema\
    .basin_height_volume_table\
    .filter(c("water_basin_fk") == water_basin_uuid)
if basin_metadata["height_min"] is not None:
    basin_height_volume_table = basin_height_volume_table.filter(c("height") >= basin_metadata["height_min"])
if basin_metadata["height_max"] is not None:
    basin_height_volume_table = basin_height_volume_table.filter(c("height") <= basin_metadata["height_max"])




In [5]:
down_stream_height

1970.0

In [6]:
discharge_volume: pl.DataFrame = generate_clean_timeseries(
    data=discharge_flow_measurement, datetime_index=first_datetime_index,
    col_name="discharge_volume", min_datetime=min_datetime, 
    max_datetime=max_datetime, time_delta=first_time_delta, agg_type="sum")

market_price: pl.DataFrame = generate_clean_timeseries(
    data=market_price_measurement, datetime_index=first_datetime_index,
    col_name="avg", min_datetime=min_datetime, 
    max_datetime=max_datetime, time_delta=first_time_delta, agg_type="mean")

basin_height : pl.DataFrame = generate_segments(
    data=basin_height_volume_table, x_col="height", y_col="volume",
    min_x=basin_metadata["height_min"], max_x=basin_metadata["height_max"], 
    n_segments=n_segments)


In [7]:
resource_fk_list = small_flex_input_schema.resource\
    .filter(c("uuid").is_in(power_plant_metadata["resource_fk_list"]))\
    .filter(c("type") == "hydro_turbine")["uuid"].to_list()
    
not_selected_resource = [idx for idx, item in enumerate(power_plant_metadata["resource_fk_list"]) if item not in resource_fk_list]

power_plant_state = small_flex_input_schema.power_plant_state.filter(c("power_plant_fk") == power_plant_uuid)
if not_selected_resource:
    power_plant_state = power_plant_state\
    .filter(
        pl.all_horizontal(c("resource_state_list").list.get(i) == False for i in not_selected_resource)
    ).with_row_index(name="index")
power_plant_state_mapping = pl_to_dict(power_plant_state[["uuid", "index"]])

power_performance_table: pl.DataFrame = small_flex_input_schema.hydro_power_performance_table.join(
    power_plant_state[["uuid", "index"]], left_on="power_plant_state_fk", right_on="uuid", how="inner"
).sort("index", "head")


In [8]:
solver = pyo.SolverFactory('gurobi')

model: pyo.AbstractModel = pyo.AbstractModel()
model.T = pyo.Set()
model.S = pyo.Set()
model.H = pyo.Set()

model.t_max = pyo.Param()
model.alpha = pyo.Param()
model.max_flow = pyo.Param() # m^3/h
model.min_flow = pyo.Param() # m^3/h
model.start_basin_height = pyo.Param()

model.max_basin_height = pyo.Param(model.H)
model.min_basin_height = pyo.Param(model.H)
model.basin_dH_dV = pyo.Param(model.H)

model.discharge_volume = pyo.Param(model.T)
model.market_price = pyo.Param(model.T)
model.nb_hours = pyo.Param(model.T)


model.basin_state = pyo.Var(model.T, model.H, within=pyo.Binary)
model.basin_height = pyo.Var(model.T, within=pyo.NonNegativeReals)

model.basin_volume_diff = pyo.Var(model.T, model.H, within=pyo.Binary)
model.V_tot = pyo.Var(model.T, within=pyo.Reals)

@model.Objective(sense=pyo.maximize) # type: ignore
def selling_income(model):
    return sum(model.market_price[t] * model.V_tot[t] * model.alpha  for t in model.T)

### Basin height constraints
@model.Constraint(model.T, model.H) # type: ignore
def basin_max_height_constraint(model, t, h):
    return model.basin_height[t] >= (1 - model.basin_state[t, h]) * model.max_basin_height[h]

@model.Constraint(model.T, model.H) # type: ignore
def basin_min_height_constraint(model, t, h):
    return model.basin_height[t] >= model.basin_state[t, h] * model.min_basin_height[h]

@model.Constraint(model.T) # type: ignore
def basin_height_evolution(model, t):
    if t == 0:
        return model.basin_height[t] == model.start_basin_height
    elif t == model.t_max:
        return model.basin_height[t] == model.start_basin_height
    else:
        return (
            model.basin_height[t + 1] == 
            model.basin_height[t] + sum(
                (model.discharge_volume[t] * model.basin_state[t, h] - model.basin_volume_diff[t, h]) * model.basin_dH_dV[h] 
                for h in model.H
            )
        )

# Basin volume constraints 
@model.Constraint(model.T, model.H) # type: ignore
def volume_max_inactive_constraint(model, t, h):
    return (
        model.basin_volume_diff[t, h] <= model.max_flow * model.nb_hours[t] * model.basin_state[t, h]
    )
    
@model.Constraint(model.T, model.H) # type: ignore
def volume_min_inactive_constraint(model, t, h):
    return (
        model.basin_volume_diff[t, h] >= model.min_flow * model.nb_hours[t] * model.basin_state[t, h]
    )

@model.Constraint(model.T, model.H) # type: ignore
def volume_max_active_constraint(model, t, h):
    return (
        model.basin_volume_diff[t, h] <= model.V_tot[t]  - model.min_flow * model.nb_hours[t] * (1 - model.basin_state[t, h])
    )

@model.Constraint(model.T, model.H) # type: ignore
def volume_min_active_constraint(model, t, h):
    return (
        model.basin_volume_diff[t, h] >= model.V_tot[t]  - model.max_flow * model.nb_hours[t] * (1 - model.basin_state[t, h])
    )
    
@model.Constraint(model.T) # type: ignore
def max_volume_constraint(model, t):
    return (
        model.V_tot[t] <= model.max_flow * model.nb_hours[t]
    )

@model.Constraint(model.T) # type: ignore
def min_volume_constraint(model, t):
    return (
        model.V_tot[t] >= model.min_flow * model.nb_hours[t]
    )



In [10]:
alpha = power_performance_table.select((c("electrical_power")/c("flow")).drop_nans().alias("alpha")).mean()["alpha"][0]
max_flow = power_performance_table.filter(c("flow") > 0)["flow"].min()*3600 # type: ignore
min_flow = 0
start_height = small_flex_input_schema.basin_height_measurement.filter(c("timestamp") == min_datetime)["height"][0]

data: dict = {}

sets: dict = {
    "T": first_datetime_index["index"].to_list(),
    "H": basin_height["index"].to_list(),
}

constant_params: dict =  {
    "t_max": first_datetime_index.height - 1,
    "alpha": alpha,
    "max_flow": max_flow,
    "min_flow": min_flow,
    "start_basin_height": start_height
}

set_params: dict = {
    "max_basin_height": pl_to_dict(basin_height[["index", "max_height"]]),
    "min_basin_height": pl_to_dict(basin_height[["index", "min_height"]]),
    "basin_dH_dV": pl_to_dict(basin_height[["index", "dx_dy"]]),
    "market_price": pl_to_dict(market_price[["index", "avg"]]),
    "nb_hours": pl_to_dict(first_datetime_index[["index", "n_index"]]),
    "discharge_volume": pl_to_dict(discharge_volume[["index", "discharge_volume"]])
}

data.update(dict(map(lambda set: (set[0], {None: set[1]}), sets.items())))
data.update(dict(map(lambda constant_param: (constant_param[0], {None: constant_param[1]}), constant_params.items())))
data.update(set_params)

model_instance = model.create_instance({None: data})
res = solver.solve(model_instance, load_solutions=True, tee=False)

model.name="unknown";
    - termination condition: infeasibleOrUnbounded
    - message from solver: Problem proven to be infeasible or unbounded.
